# Farmer problem

Ref: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

In [5]:
using JuMP

In [6]:
#using Gurobi

In [7]:
using Clp

In [8]:
#Pkg.add("Gurobi")

# Average Scenario

In [9]:
m = Model(solver = ClpSolver())

crops = ["Wheat", "Corn", "Suger beets"]
ncrops = length(crops)

@variable(m,x[1:ncrops] >= 0)
@variable(m,y[1:2] >= 0)
@variable(m,w[1:4] >= 0)

costSeeding = [150, 230, 260]
costs = [238, 210]
prices = [170, 150, 36, 10]

4-element Array{Int64,1}:
 170
 150
  36
  10

In [10]:
typeof(m)

JuMP.Model

In [11]:
@constraint(m, surface, sum(x[i] for i=1:3) <= 500)

x[1] + x[2] + x[3] ≤ 500

In [12]:
@constraint(m, wheatNeeds, 2.5x[1]+y[1]-w[1] >= 200)

2.5 x[1] + y[1] - w[1] ≥ 200

In [13]:
@constraint(m, cornNeeds, 3x[2]+y[2]-w[2] >= 240)

3 x[2] + y[2] - w[2] ≥ 240

In [14]:
@constraint(m, beetsProd, w[3]+w[4] <= 20x[3])

w[3] + w[4] - 20 x[3] ≤ 0

In [15]:
@constraint(m, beetsQuota, w[3] <= 6000)

w[3] ≤ 6000

In [16]:
@objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))

150 x[1] + 230 x[2] + 260 x[3] + 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]

In [17]:
print(m)

Min 150 x[1] + 230 x[2] + 260 x[3] + 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]
Subject to
 x[1] + x[2] + x[3] ≤ 500
 2.5 x[1] + y[1] - w[1] ≥ 200
 3 x[2] + y[2] - w[2] ≥ 240
 w[3] + w[4] - 20 x[3] ≤ 0
 w[3] ≤ 6000
 x[i] ≥ 0 ∀ i ∈ {1,2,3}
 y[i] ≥ 0 ∀ i ∈ {1,2}
 w[i] ≥ 0 ∀ i ∈ {1,2,3,4}


In [18]:
solve(m)

:Optimal

In [19]:
print("Solution: $(getvalue(x))")

Solution: [120.0, 80.0, 300.0]

We will summarize the solution approach using a Julia function.

In [20]:
function farmer(factor::Float64 = 1.0, m::Model = Model(solver = ClpSolver()))
    # m = Model(solver = ClpSolver())
    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)
    @variable(m,y[1:2] >= 0)
    @variable(m,w[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = factor*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNeeds, returns[1]x[1]+y[1]-w[1] >= 200)
    @constraint(m, cornNeeds, returns[2]x[2]+y[2]-w[2] >= 240)
    @constraint(m, beetsProd, w[3]+w[4] <= returns[3]x[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = solve(m)
    println(getvalue(x))
    return m, status
end

farmer (generic function with 3 methods)

In [21]:
maverage, status = farmer()

[120.0, 80.0, 300.0]


(Minimization problem with:
 * 5 linear constraints
 * 9 variables
Solver is ClpMathProg, :Optimal)

In [22]:
maverage

Minimization problem with:
 * 5 linear constraints
 * 9 variables
Solver is ClpMathProg

In [23]:
getobjectivevalue(maverage)

-118600.0

## Good scenario

In [24]:
mgood, status = farmer(1.2)

[183.333, 66.6667, 250.0]


(Minimization problem with:
 * 5 linear constraints
 * 9 variables
Solver is ClpMathProg, :Optimal)

In [25]:
mgood

Minimization problem with:
 * 5 linear constraints
 * 9 variables
Solver is ClpMathProg

In [26]:
getobjectivevalue(mgood)

-167666.6666666667

## Bad scenario

In [28]:
mbad, status = farmer(0.8, Model(solver = ClpSolver()))

[100.0, 25.0, 375.0]


(Minimization problem with:
 * 5 linear constraints
 * 9 variables
Solver is ClpMathProg, :Optimal)

In [29]:
getobjectivevalue(mbad)

-59950.0

## Stochastic program - extended form

We have to combine the three scenarios. We will assume that each one has a probability equal to 1/3.

In [30]:
p = [1/3, 1/3, 1/3]

3-element Array{Float64,1}:
 0.333333
 0.333333
 0.333333

In [31]:
function farmerStoch()
    m = Model(solver = ClpSolver())

    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)

    @variable(m,ya[1:2] >= 0)
    @variable(m,wa[1:4] >= 0)

    @variable(m,yg[1:2] >= 0)
    @variable(m,wg[1:4] >= 0)

    @variable(m,yb[1:2] >= 0)
    @variable(m,wb[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returnsA = [2.5, 3, 20]
    returnsG = 1.2*[2.5, 3, 20]
    returnsB = 0.8*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNA, returnsA[1]x[1]+ya[1]-wa[1] >= 200)
    @constraint(m, cornNA, returnsA[2]x[2]+ya[2]-wa[2] >= 240)
    @constraint(m, beetsPA, wa[3]+wa[4] <= returnsA[3]x[3])
    @constraint(m, beetsQA, wa[3] <= 6000)

    @constraint(m, wheatNG, returnsG[1]x[1]+yg[1]-wg[1] >= 200)
    @constraint(m, cornNG, returnsG[2]x[2]+yg[2]-wg[2] >= 240)
    @constraint(m, beetsPG, wg[3]+wg[4] <= returnsG[3]x[3])
    @constraint(m, beetsQG, wg[3] <= 6000)

    @constraint(m, wheatNB, returnsB[1]x[1]+yb[1]-wb[1] >= 200)
    @constraint(m, cornNB, returnsB[2]x[2]+yb[2]-wb[2] >= 240)
    @constraint(m, beetsPB, wb[3]+wb[4] <= returnsB[3]x[3])
    @constraint(m, beetsQB, wb[3] <= 6000)

    @objective(m, Min, sum(costSeeding[i]*x[i] for i=1:3)
        + 1/3*sum(costs[i]*ya[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wa[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yg[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wg[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yb[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wb[i] for i=1:length(prices)))
    
    status = solve(m)

    return m, status
end

farmerStoch (generic function with 1 method)

In [32]:
mstoch, status = farmerStoch()

(Minimization problem with:
 * 13 linear constraints
 * 21 variables
Solver is ClpMathProg, :Optimal)

In [33]:
valStoch = getobjectivevalue(mstoch)

-108390.00000000003

## Expected value of the perfect information

In [34]:
solutions = [getobjectivevalue(maverage), getobjectivevalue(mgood), getobjectivevalue(mbad)]

3-element Array{Float64,1}:
 -118600.0      
      -1.67667e5
  -59950.0      

Expected revenue under perfect information

In [35]:
valPerfect = p'*solutions

-115405.55555555555

Expected value of perfect information

In [36]:
valPerfect - valStoch

-7015.555555555518

## Value of the stochastic solution

We have to fix the first stage decision. We define the second stage problem.

In [40]:
function secondStage(x, factor::Float64 = 1.0)
    m = Model(solver = ClpSolver())

    @variable(m,y[1:2] >= 0)
    @variable(m,w[1:4] >= 0)

    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = (factor*[2.5, 3, 20]).*x
    z = [200 - returns[1], 240 - returns[2], returns[3]]
    
    @constraint(m, wheatNeeds, y[1]-w[1] >= z[1])
    @constraint(m, cornNeeds, y[2]-w[2] >= z[2])
    @constraint(m, beetsProd, w[3]+w[4] <= z[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min, sum(costs[i]*y[i] for i = 1:length(costs)) - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = solve(m)
    
    return m, status
end

secondStage (generic function with 2 methods)

In [41]:
x = [120, 80, 300]

3-element Array{Int64,1}:
 120
  80
 300

In [42]:
msecond, status = secondStage(x)

(Minimization problem with:
 * 4 linear constraints
 * 6 variables
Solver is ClpMathProg, :Optimal)

In [43]:
msecond

Minimization problem with:
 * 4 linear constraints
 * 6 variables
Solver is ClpMathProg

In [44]:
vaverage = getobjectivevalue(msecond)

-233000.0

In [45]:
msecond, status = secondStage(x, 1.2)

(Minimization problem with:
 * 4 linear constraints
 * 6 variables
Solver is ClpMathProg, :Optimal)

In [46]:
vgood = getobjectivevalue(msecond)

-262400.0

In [47]:
msecond, status = secondStage(x, 0.8)

(Minimization problem with:
 * 4 linear constraints
 * 6 variables
Solver is ClpMathProg, :Optimal)

In [48]:
vbad = getobjectivevalue(msecond)

-169520.0

Expected revenue

In [49]:
er = costSeeding'*x+p'*[vaverage, vgood, vbad]

-107239.99999999997

Value of the stochastic solution

In [50]:
vss = valStoch - er

-1150.0000000000582